In [1]:
from PIL import Image, ImageDraw, ImageFont
import random
import cv2
import os
import matplotlib.pyplot as plt

In [2]:
def infinte_cube(img):
    ''' Perform atomic operations on image similar to infinity cube. On performing these steps 'n' times, we can generate the biometric template.
        Input:
            img: The input image
        Output:
            returns the image after performing atomic operation once
    '''
    # Make width dimension even, if it is not
    if img.shape[1]%2 != 0:
        img = img[:, 0:img.shape[1]-1, :]    

    # Expand horizontally
    new_img = cv2.resize(img, (2*img.shape[1], img.shape[0]))

    # Make the height and width of even number
    if new_img.shape[0]%2 != 0:
        new_img = new_img[0:-2, :, :]
    if new_img.shape[1]%2 != 0:
        new_img = new_img[:, 0:-2, :]
#     plt.imshow(new_img)
#     plt.show()
#     new_img.shape

    # Split image vertically into top image and bottom image
    height, width, channels = new_img.shape
    half_height = height//2

    top_img, bottom_img = new_img[:half_height, :], new_img[half_height:, :]

#     plt.imshow(top_img)
#     plt.show()
#     plt.imshow(bottom_img)
#     plt.show()
#     top_img.shape, bottom_img.shape

    # Perform XOR operation between top and bottom section of images
    XOR_img = np.bitwise_xor(top_img, bottom_img)

#     plt.imshow(XOR_img)

    # Split image into 4 equal parts
    height, width, channels = XOR_img.shape
    quarter_width = width//4
    if width%4 != 0:
        quarter_width = int(width//4)
        XOR_img = XOR_img[:, 1:XOR_img.shape[1]-1, :]

    img1, img2, img3, img4 = XOR_img[:, 0:quarter_width], XOR_img[:, quarter_width: 2*quarter_width], XOR_img[:, 2*quarter_width:3*quarter_width], XOR_img[:, 3*quarter_width:]

#     plt.imshow(img1)
#     plt.show()
#     plt.imshow(img2)
#     plt.show()
#     plt.imshow(img3)
#     plt.show()
#     plt.imshow(img4)
#     plt.show()

    # Create new top and bottom images by merging middle two images and two outermost images respectively 
    top_img = cv2.hconcat([img2, img3])
    bottom_img = cv2.hconcat([img1, img4])

#     plt.imshow(top_img)
#     plt.show()
#     plt.imshow(bottom_img)
#     plt.show()
    
    # Form the image of originaly shape by concatenating the two images vertically
#     print(top_img.shape)
#     print(bottom_img.shape)
    new_img = cv2.vconcat([top_img, bottom_img])

#     plt.imshow(new_img)
#     new_img.shape
    
    # Expand vertically
    new_img = cv2.resize(new_img, (new_img.shape[1], 2*new_img.shape[0]))

#     plt.imshow(new_img)
#     plt.show()
#     new_img.shape

    # Split image horizontally into left image and right image
    height, width, channels = new_img.shape
    half_width = width//2

    left_img, right_img = new_img[:, :half_width], new_img[:, half_width:]

#     plt.imshow(left_img)
#     plt.show()
#     plt.imshow(right_img)
#     plt.show()
#     left_img.shape, right_img.shape

    # Perform XOR operation between left and right section of images
    XOR_img = np.bitwise_xor(left_img, right_img)

#     plt.imshow(XOR_img)

    # Split image into 4 equal parts
    height, width, channels = XOR_img.shape
    quarter_height = height//4

    img1, img2, img3, img4 = XOR_img[0:quarter_height, :], XOR_img[quarter_height:2*quarter_height, :], XOR_img[2*quarter_height:3*quarter_height, :], XOR_img[3*quarter_height:, :]

#     plt.imshow(img1)
#     plt.show()
#     plt.imshow(img2)
#     plt.show()
#     plt.imshow(img3)
#     plt.show()
#     plt.imshow(img4)
#     plt.show()

    # Create new left and right images by merging middle two images and two outermost images respectively 
    left_img = cv2.vconcat([img2, img3])
    right_img = cv2.vconcat([img1, img4])

#     plt.imshow(left_img)
#     plt.show()
#     plt.imshow(right_img)
#     plt.show()

    # Form the image of originaly shape by concatenating the two images vertically
    new_img = cv2.hconcat([left_img, right_img])
#     plt.imshow(new_img)
#     new_img.shape

    return new_img

# # Generate template for a given image
# def inf_cube_with_itr(icwi_img, iterations):
#     ''' Create a template for the given image with specified number of iterations
#         Input:
#             icwi_img: an image for which we want to generate the template
#             iterations: the nuber of times we want to perform infinty cube transforamtions to generate the tempalte
#         Output:
#             returns the generated template for the given input image
#     '''
#     icwi_template = icwi_img.copy()
#     for _ in range(iterations):
#         icwi_template = infinte_cube(icwi_template)
#     return icwi_template


# # Generate tempalte for whole batch at once
# def gen_batch_template(gbt_imgs):
#     ''' Generate a set of templates for the given set of images
#         Input:
#             gbt_imgs: a list of input images
#         Output:
#             returns the generated template for every image present in the input set
#     '''
#     gbt_temp = []
#     for gbt_img_temp in gbt_imgs:
#         gbt_img_temp = np.array(gbt_img_temp, dtype= 'uint8')
#         gbt_img_temp = inf_cube_with_itr(gbt_img_temp, iterations)
#         gbt_temp.append(gbt_img_temp)
#     return gbt_temp

In [81]:
def num_to_color(xor_num):
    # Simple mapping function to convert XOR result to RGB values
    r = xor_num % 256
    g = (xor_num // 256) % 256
    b = (xor_num // 65536) % 256
    return r, g, b

def draw_color_maze(image, draw, maze_size, color, user_id, number, savegif = False):
    # Set a fixed seed for the random number generator based on user ID and number
    random.seed(hash((user_id, number)))
    
    if savegif == True:
        counter = 1
    # Use the chosen color for the maze walls
    for i in range(0, maze_size[0], 10):
        for j in range(0, maze_size[1], 10):
            if random.random() < 0.4:  # Adjust this probability to control maze density
                draw.rectangle([i, j, i + 10, j + 10], fill=color)
                if savegif == True:
                    file_name = os.path.join(os.getcwd(), 'maze_steps', f'{counter}.png')
                    image.save(file_name)
                    counter += 1

def create_color_maze_image_with_xor(user_id, number, output_path='color_maze_image.png', maze_size=(252, 252), font_size=30):
    # XOR the user ID and the number
    xor_result = user_id ^ number

    # Map the XOR result to RGB values
    bg_color = num_to_color(xor_result)
    maze_color = num_to_color(user_id + 50)
    
    # Create a blank white image
    image = Image.new('RGB', maze_size, bg_color)
    draw = ImageDraw.Draw(image)

    # Draw the color maze pattern
    draw_color_maze(image, draw, maze_size, maze_color, user_id, number, savegif=True)

    # Save the image
    image.save(output_path)

In [84]:
# # Create GIF

# import glob

# frames = []
# create_color_maze_image_with_xor(user_id=202211001, number=435435, output_path='4final.png')

# for index, _ in enumerate(os.listdir(os.path.join(os.getcwd(), 'maze_steps')), start=1):
# #     print(index)
#     img = Image.open(os.path.join(os.getcwd(), 'maze_steps', f'{index}.png'))
#     frames.append(img)

# frame_one = frames[0]
# frame_one.save("maze_making.gif", format="GIF", append_images=frames, save_all=True, duration=50, loop=0)

In [39]:
user_id = 202211000
with open('biohash_numbers.txt', 'r') as f:
    for biohash_index, line in enumerate(f.readlines(), start=user_id):
        
        # Read the biohash number
        biohash_number = line
        biohash_number = int(biohash_number.split('\n')[0])
        # long_biohash_number = ''.join([biohash_number]*4)
        
        # print(biohash_number, biohash_index)
        
        dir_name = os.path.join(os.getcwd(), 'datasets', 'maze-like-img', str(biohash_number))
        if not os.path.exists(dir_name):
            os.makedirs(dir_name)
        file_name = os.path.join(dir_name, '1.png')

        create_color_maze_image_with_xor(user_id=biohash_index, number=biohash_number, output_path=os.path.join(dir_name, file_name))

In [58]:
import numpy as np
dataset_dir = os.path.join(os.getcwd(), 'datasets', 'maze-like-img')
output_dir = os.path.join(os.getcwd(), 'datasets', 'maze-like-img-XOR')

for folder_name in os.listdir(dataset_dir):
    folder_path = os.path.join(dataset_dir, folder_name)
    output_folder_path = os.path.join(output_dir, folder_name)
    
    if not os.path.exists(output_folder_path):
        os.makedirs(output_folder_path)
        
    for file_name in os.listdir(folder_path):
        img = cv2.imread(os.path.join(folder_path, file_name))
#         plt.imshow(img)
#         plt.show()
        
        XOR_img = infinte_cube(img)
#         plt.imshow(XOR_img)
#         plt.axis('off')
#         plt.show()
        cv2.imwrite(os.path.join(output_folder_path, file_name), XOR_img)